In [13]:
import pandas as pd
from ast import literal_eval

In [15]:
import sys
import csv
import struct
platform_c_maxint = 2 ** (struct.Struct('i').size * 8 - 1) - 1
csv.field_size_limit(platform_c_maxint)
files = [r'C:\Users\huda\Downloads\results_ortools.txt', r'C:\Users\huda\Downloads\results_cplex_cp.txt', r'C:\Users\huda\Downloads\results_gurobi.txt', r'C:\Users\huda\Downloads\results_hexaly_rewritten.txt', r'C:\Users\huda\Downloads\results_cplex_lp_rewritten.txt']


In [111]:
from ast import literal_eval
import math
data = dict()
instances = []
for file in files:
    input_file = csv.DictReader(open(file), delimiter=';', fieldnames=['name','optimization_status','fitness_value','lower_bound', 'runtime', 'result_vector1', 'result_vector2', 'result_vector3', 'peak_cpu', 'peak_ram', 'resource_history', 'best_result_history'])
    in_data = []
    for row in input_file:
        name = row['name'][:-12]
        if name.startswith('Error'):
            continue
        if name not in instances:
            instances.append(name)
        fitness = int(float(row['fitness_value']) + 0.5) if row['fitness_value'] and row['fitness_value'].isnumeric() else float('inf')#int(float(row['fitness_value']) + 0.5)
        lb = int(float(row['lower_bound']) + 0.5) if row['lower_bound'] and row['lower_bound'].isnumeric() else float('inf')#int(float(row['lower_bound']) + 0.5)
        if not math.isinf(lb) and not math.isinf(fitness):
            #progress_history = [r[2:] for r in row['best_result_history'].split('),')[1:]]#literal_eval(row['best_result_history'])
            #progress_history = [[float(x.split(',')[0]), int(float(x.split(',')[1][1:]) + 0.5), int(float(x.split(',')[2][1:])) if x.split(',')[2][1:].isnumeric() else int(float(x.split(',')[2][1:-2]))] for x in progress_history]
            progress_history = [[float(x.split(',')[0]), int(float(x.split(',')[1][1:]) + 0.5), int(float(x.split(',')[2][1:])) if x.split(',')[2][1:].isnumeric() else int(float(x.split(',')[2][1:-2]))] for x in [r[2:] for r in row['best_result_history'].split('),')[1:]]]
            progress_history.append((float(row['runtime']), fitness, lb)) # in case callbacks didn't get triggered for the final result
            solver = file.split('\\')[-1][:-4]
            if solver == 'results_hexaly_rewritten':
                solver = 'results_hexaly'
            elif solver == 'results_cp_lp_rewritten':
                solver = 'results_cp_lp'
            solver = solver[8:]
            if solver not in data:
                data[solver] = dict()
            data[solver][name] = [fitness, lb, progress_history]
    

In [117]:
data.keys()

dict_keys(['cplex_cp', 'hexaly'])

In [113]:
# fitness
max = False
col = 1
# lower bound
#max = True
#col = 2
targets = dict()
for instance in instances:
    best = -float('inf') if max else float('inf')
    if instance not in targets:
        targets[instance] = dict()
    for key in data.keys():
        if instance in data[key]:
            if (max and data[key][instance][col] > best) or (not max and data[key][instance][col] < best):
                best = data[key][instance][col]
            targets[instance][key] = data[key][instance][col]
    targets[instance]['overall'] = best

In [114]:
max_diff = 0.1
results = dict()
for instance in targets.keys():
    target_overall = targets[instance]['overall']
    if max:
        target_overall *= (1.0+max_diff)
    else:
        target_overall *= (1.0-max_diff)
    if instance not in results:
        results[instance] = dict()
    for solver in targets[instance].keys():
        if solver == 'overall':
            continue
        target_self = targets[instance][solver]
        if max:
            target_self *= (1.0+max_diff)
        else:
            target_self *= (1.0-max_diff)
        if solver not in results[instance]: # should never be in there?
            results[instance][solver] = dict()
        for entry in data[solver][instance][2]:
            if (max and entry[col] >= target_self) or (not max and entry[col] <= target_self):
                results[instance][solver]['self'] = entry[0] # timestamp
            if (max and entry[col] >= target_overall) or (not max and entry[col] <= target_overall):
                results[instance][solver]['overall'] = entry[0]

In [115]:
results

{'0_BehnkeGeiger_10': {'cplex_cp': {}, 'hexaly': {}},
 '0_BehnkeGeiger_11': {'cplex_cp': {}, 'hexaly': {}},
 '0_BehnkeGeiger_12': {'cplex_cp': {}, 'hexaly': {}},
 '0_BehnkeGeiger_13': {'cplex_cp': {},
  'hexaly': {'self': 1.0, 'overall': 1.0}},
 '0_BehnkeGeiger_14': {'cplex_cp': {}, 'hexaly': {}},
 '0_BehnkeGeiger_15': {'cplex_cp': {}, 'hexaly': {}},
 '0_BehnkeGeiger_16': {'cplex_cp': {},
  'hexaly': {'self': 2.0, 'overall': 2.0}},
 '0_BehnkeGeiger_17': {'cplex_cp': {},
  'hexaly': {'self': 2.0, 'overall': 2.0}},
 '0_BehnkeGeiger_18': {'cplex_cp': {},
  'hexaly': {'self': 2.0, 'overall': 2.0}},
 '0_BehnkeGeiger_19': {'cplex_cp': {},
  'hexaly': {'self': 2.0, 'overall': 2.0}},
 '0_BehnkeGeiger_1': {'cplex_cp': {}, 'hexaly': {}},
 '0_BehnkeGeiger_20': {'cplex_cp': {},
  'hexaly': {'self': 2.0, 'overall': 2.0}},
 '0_BehnkeGeiger_21': {'cplex_cp': {}, 'hexaly': {}},
 '0_BehnkeGeiger_22': {'cplex_cp': {}, 'hexaly': {}},
 '0_BehnkeGeiger_23': {'cplex_cp': {}, 'hexaly': {}},
 '0_BehnkeGeiger_